In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import re
import functools
from time import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import utils
import data_reader
from batch_norm import Batch_Normalizer
from model import CNN_Model

In [2]:
# File parameters
DATA_DIR = 'data'
TRAIN_IMAGES = 'data_train.bin'
TRAIN_TEMPLATES = 'fv_train.bin'
VALID_IMAGES = 'data_valid.bin'
VALID_TEMPLATES = 'fv_valid.bin'
TEST_IMAGES = 'data_test.bin'

# Data parameters
NUM_TRAIN_IMAGES = 100000
NUM_VALID_IMAGES = 10000
NUM_TEST_IMAGES = 10000
IMAGE_DIM = 48
TEMPLATE_DIM = 128

# Reader parameters
BATCH_SIZE = 100
VALID_BATCH_SIZE = 1000
NUM_READ_THREADS = 1
MIN_AFTER_DEQUEUE = 10000
CAPACITY = 11000

# Batch norm parameters
EMA_DECAY = 0.99
BN_EPSILON = 0.01

# Log parameters
LOG_DIR = 'logs'
SAVE_DIR = 'checkpoints'

In [21]:
def test_config():

    model = CNN_Model('config2', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
                      VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

    model.add_conv('conv_1_1', ksize=[3, 3, 1, 10])
    model.add_batch_norm('bn_1_1')
    model.add_conv('conv_1_2', ksize=[3, 3, 10, 10])
    model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_1_2')

    model.add_conv('conv_2_1', ksize=[3, 3, 10, 20])
    model.add_batch_norm('bn_2_1')
    model.add_conv('conv_2_2', ksize=[3, 3, 20, 20])
    model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_2_2')

    model.add_conv('conv_3_1', ksize=[3, 3, 20, 40])
    model.add_batch_norm('bn_3_1')
    model.add_conv('conv_3_2', ksize=[3, 3, 40, 40])
    model.add_pool('max_pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_3_2')

    model.add_pool('avg_pool', 'avg', ksize=[1, 3, 3, 1], stride=[1, 3, 3, 1])
    model.add_fully_connected('fc', size=128)

    model.add_mse_loss('mse_loss')
    model.add_adam_optimizer('optimizer', init_learning_rate=0.0001, decay=False)
    model.add_summaries('summaries')

    
#     utils.add_description("""
#     model = CNN_Model('config2', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
#                       VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

#     model.add_conv('conv_1_1', ksize=[3, 3, 1, 10])
#     model.add_batch_norm('bn_1_1')
#     model.add_conv('conv_1_2', ksize=[3, 3, 10, 10])
#     model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_1_2')

#     model.add_conv('conv_2_1', ksize=[3, 3, 10, 20])
#     model.add_batch_norm('bn_2_1')
#     model.add_conv('conv_2_2', ksize=[3, 3, 20, 20])
#     model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_2_2')

#     model.add_conv('conv_3_1', ksize=[3, 3, 20, 40])
#     model.add_batch_norm('bn_3_1')
#     model.add_conv('conv_3_2', ksize=[3, 3, 40, 40])
#     model.add_pool('max_pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_3_2')

#     model.add_pool('avg_pool', 'avg', ksize=[1, 3, 3, 1], stride=[1, 3, 3, 1])
#     model.add_fully_connected('fc', size=128)

#     model.add_mse_loss('mse_loss')
#     model.add_adam_optimizer('optimizer', init_learning_rate=0.01, decay=False)
#     model.add_summaries('summaries')
    
#     total parameters = %d
#     """  % (utils.num_parameters(model)), model.save_dir, model.name)
    
    
    model.initialize_session(restore=True)
    model.train(n_batches=5000, step_size=500, batch_size=100, valid_batch_size=1000, save=True)
    model.close_session()

In [ ]:
test_config()

# L2 Regularization